In [1]:
# Importing libraries and dataset
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from torch.utils.data import Dataset, DataLoader
from torch import nn
from keras.datasets import mnist
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
train_dataset = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)
test_dataset = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor(),
    download=True
)

In [3]:
train_dataloader = DataLoader(dataset=train_dataset,batch_size=100,shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset,batch_size=100,shuffle=True)

In [4]:
class net(nn.Module):
  def __init__(self,input_size,output_size):     # defining the layers 
    super(net,self).__init__()
    self.flatten = nn.Flatten()
    self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
    self.conv2 = nn.Sequential(         
        nn.Conv2d(16, 32, 5, 1, 2),     
        nn.ReLU(),                      
        nn.MaxPool2d(2),                
    )
    self.out = nn.Linear(32*7*7, output_size)

  def forward(self,x):    # defining the forward pass which is invoked automatically when the class is called
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.flatten(x)
    output = self.out(x)
    return output

In [5]:
input_size = 28*28
output_size = 10

model = net(input_size,output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.001)

epochs = 10

In [6]:
cost_values = []
loss_values = []

for epoch in range(epochs):
    for i, (x_train, y_train) in enumerate(train_dataloader):
        
        y_pred = model(x_train)
        y_train = y_train.long()
        loss = criterion(y_pred, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [7]:
model.eval()
size = len(test_dataloader.dataset)
num_batches = len(test_dataloader)
test_loss, correct = 0, 0

with torch.no_grad():
    for i, (x_test, y_test) in enumerate(test_dataloader):
        pred = model(x_test)
        test_loss += criterion(pred, y_test).item()
        correct += (pred.argmax(1) == y_test).type(torch.float).sum().item()

test_loss /= num_batches
correct /= size
print(f"Accuracy: {(100*correct):>0.2f}%\n")

Accuracy: 92.17%

